# CSL - List
## Achtung
* **Eine Zelle in PDF war verbunden - diese Zeile hatte verschobene Totale. Überprüfen!**

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib
import re

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [3]:
# Read pdf into DataFrame
df = tabula.read_pdf("Offenlegung 2019 HCP_HCO.pdf", pages='all', lattice=True)

## Format Table

In [4]:
df_export = df.copy()

#Rename Columns
df_export.columns = ['empty0', 'name', 'location', 'country', 'address', 'uci', 'donations_grants', 'sponsorship', 'registration_fees','travel_accommodation', 'fees', 'related_expenses', 'empty1', 'total']

#Shift Rows
df_export[df_export['empty0'].notna()] = df_export.shift(1, axis='columns')

#Add Type
index_hco = df_export[df_export['name'].str.contains("Aggregate amount", na=False)].index[0]
df_export['type'] = np.where(df_export.index < index_hco, 'hcp', 'hco')

#Remove empty
df_export.drop(columns=['empty0', 'empty1'], inplace=True)

#Remove, where country not Switzerland
df_export = df_export[df_export.country == 'Switzerland']

#Convert to Numbers and Sum
df_export['total'] = '0'
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#Add PLZ
df_export = add_plz(df_export)

#Remove Titles
df_export['name'] = df_export.name.str.replace(regex_title, '', regex=True, case=False)
df_export['name'] = df_export.name.str.strip()

#Remove Carinations
df_export = remove_carination(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

export_list(df_export, 'csl')

saved


In [5]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)